In [1]:
USE_SERVER = True
if USE_SERVER:
    !pip install elasticsearch
    !pip install elasticsearch_dsl
    !pip install pymed
    !pip install gensim
    !pip install torchsummary

     |████████████████████████████████| 188 kB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 24.2 MB 35.9 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 105.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110317 sha256=153599aaccc5add74356e9f4ef6576225133de2477e8af09a93b5c65aaad2de7
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a4/9b/d5/85705a7ab783cd6f7bd718f01d3b1396272f30044e3c36401a
Successfully built smart-open


In [2]:
from keras.layers import Dense, Embedding, \
Input, Lambda, RepeatVector, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
from __future__ import print_function
import argparse
import numpy as np

import pandas as pd
import utils
import os
from utils import PROJECT_ROOT, DATA_PATH
import yuval_module.paper_source as PaperSource

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import py_4.get_mesh_vec as get_mesh_vec
import py_3.sim_matrix_3 as sim_matrix_3
import py_4.get_all_features as get_all_features 


from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time

from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

In [4]:
FILE = "enriched_labeled_dataset_large" 
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset(FILE)
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

PULLING FROM S3
FILE PULLED


In [5]:
cd ..

/home/ubuntu/AYP_b


In [6]:
auth_core = pd.read_csv("data/train_set_author_names.csv")["0"]
auth_eps = pd.read_csv("data/val_set_author_names.csv")["0"]
auth_usecase = pd.read_csv("data/test_set_author_names.csv")["0"]

selection_train = list(set(df['last_author_name']) - set(auth_usecase))[:4000]
selection_test = list(set(df['last_author_name']) - set(auth_usecase))[4000:4500]

df_nonan = df[np.invert(df['mesh'].isna())]

In [7]:
vae_features = get_all_features.VAE_Features(df_nonan)

'/home/ubuntu/AYP/code/models/names_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.
'/home/ubuntu/AYP/code/models/co_authors_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.


### TF DATASET

In [8]:
train_examples = vae_features.get_all_features(df_nonan[df_nonan['last_author_name'].isin(selection_train)])

X_train = tf.convert_to_tensor(train_examples)

train_dataset = tf.data.Dataset.from_tensor_slices(train_examples)

/home/ubuntu/AYP_b/code/py_4/get_all_features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['co_authors']=df.authors.apply( lambda x: [i['name'] for i in x] )


Defining new scaler


In [9]:
int(X_train.shape[0])

20012

In [10]:
def sent_generator(X_train, batchsize):
    # Create iterator that reads dataset file batch by batch 
    #your code here
    #Suffle the data file
#     indexes = np.random.choice (range(len(full)),len(full),False)
    
#     shuffled = full.iloc[indexes]

    reader = [X_train[batchsize*i: batchsize*i +batchsize,:] for i in range(int(np.ceil(int(X_train.shape[0])//batchsize)))]

    
    for df in reader:
        yield [tf.cast(df,dtype = tf.float32), tf.cast(df,dtype = tf.float32)] 
        
print([a for a in sent_generator(train_examples,  32)][:1])

[[<tf.Tensor 'Cast:0' shape=(32, 64) dtype=float32>, <tf.Tensor 'Cast_1:0' shape=(32, 64) dtype=float32>]]


### TF MODEL

In [22]:
num_sample = train_examples.shape[0]
input_dim = train_examples.shape[1]
hidden_dim = int(np.sqrt(input_dim))
n_z = 8

batch_size = 64
n_epoch = 100
m = 64

In [23]:
print("The number of samples: ", num_sample)
print("The input dim: ", input_dim)

The number of samples:  20012
The input dim:  64


In [24]:
inputs = Input(shape=(input_dim,))
h_q = Dense(hidden_dim, activation='relu')(inputs)
mu = Dense(n_z, activation='linear')(h_q)
log_sigma = Dense(n_z, activation='linear')(h_q)

In [25]:
def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(m, n_z), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps


# Sample z ~ Q(z|X)
z = Lambda(sample_z)([mu, log_sigma])

In [26]:
# P(X|z) -- decoder
decoder_hidden = Dense(hidden_dim, activation='relu')
decoder_out = Dense(input_dim, activation='linear')

h_p = decoder_hidden(z)
outputs = decoder_out(h_p)

In [27]:
def zero_loss(y_true, y_pred):
    # Return tensor filled with ones with shape equal generated sequence shape
    return K.zeros_like(y_pred)

In [28]:
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        # Create tensor (batch_size, max_sequence_len) filled with ones to consider all elements of generated sequence 
        self.target_weights = tf.constant(np.ones((batch_size, input_dim)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        # Compute sequence reconstruction loss
        recon = tf.keras.losses.MSE(x,x_decoded_mean)
        # Compute KL-divergence as Variational loss 
        kl_loss = -0.5 * K.sum(1 + log_sigma - K.square(mu) - K.exp(log_sigma), axis=-1)
        # Composite loss (reconstruction loss + Variational loss)
        return recon + kl_loss

    def call(self, inputs):
        x = inputs[0] # input sequence
        x_decoded_mean = inputs[1] # reconstructed sequence
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean) # Compute loss of the model
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape
        return K.ones_like(x)

In [41]:
# Create custom layer for loss computing
loss_layer = CustomVariationalLayer()([inputs, outputs])

vae = Model(inputs, [loss_layer])
# Use Adam optimizer with learning rate = 0.01
opt = Adam(lr=0.01)
vae.compile(optimizer=opt, loss=[zero_loss])
# Show model structure
vae.summary()

(?, 64) (64, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 8)            520         input_3[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 8)            72          dense_9[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 8)            72          dense_9[0][0]                    
____________________________________________________________________________________________

In [39]:
def create_model_checkpoint(model_name):
    filepath = f"models/{model_name}.h5"
    directory = os.path.dirname(filepath)
    try:
        # Check if directory exists
        os.stat(directory)
    except:
        # If directory doesn't exist, create the directory
        os.mkdir(directory)
    # Save model states
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=False)
    return checkpointer

# Create model checkpointer
checkpointer = create_model_checkpoint('vae')

In [40]:
nb_epoch = 25 # number of epochs for model training
n_steps = X_train.shape[0] // batch_size # Number of steps per epoch
for counter in range(nb_epoch):
    print('-------epoch: ', counter, '-------')
    # Train model. Test and save model after every epoch
    vae.fit_generator(sent_generator(X_train, batch_size),
                      steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer])
vae.save(r'vae_8dimls.h5')

-------epoch:  0 -------



Epoch 1/1


TypeError: len is not well defined for symbolic Tensors. (Cast_1250:0) Please call `x.shape` rather than `len(x)` for shape information.

In [44]:
vae.load_weights("./code/vae_8dimls.h5")

# RUN TEST EXAMPLE

In [49]:
mesh_cases = pd.read_csv('./data/mesh_data/mesh_paper_groups_pmid.csv')
typeA = mesh_cases['A']
typeB = mesh_cases['B']
df_a = pd.DataFrame((typeA,np.zeros((len(typeA),),dtype=int)),index=['pmid','class']).T
df_b = pd.DataFrame((typeB,np.ones((len(typeB),),dtype=int)),index=['pmid','class']).T
df_ab = pd.concat([df_a,df_b]).reset_index(drop=True)

In [50]:
df_ab.drop_duplicates(keep='first',inplace=True)
df_group = df[df['pmid'].isin(df_ab.pmid)]

In [53]:
usecase_examples = vae_features.get_all_features(df_group)
X_train = tf.convert_to_tensor(usecase_examples)

Using old scaler


/home/ubuntu/AYP_b/code/py_4/get_all_features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['co_authors']=df.authors.apply( lambda x: [i['name'] for i in x] )
